Tools

In [23]:
from langchain_core.tools import tool

@tool
def add(a: float, b: float) -> float:
    """Add 'a' and 'b'. Both 'a' and 'b' are numbers."""
    return a + b

@tool
def multiply(a: float, b: float) -> float:
    """Multiply 'a' and 'b'. Both 'a' and 'b' are numbers."""
    return a * b    

@tool
def subtract(a: float, b: float) -> float:
    """Subtract 'b' from 'a'. Both 'a' and 'b' are numbers."""
    return a - b

@tool
def divide(a: float, b: float) -> float:
    """Divide 'a' by 'b'. Both 'a' and 'b' are numbers."""
    return a / b

@tool
def exponentiate(a: float, b: float) -> float:
    """Raise 'a' to the power of 'b'. Both 'a' and 'b' are numbers."""
    return a ** b

tools = [add, multiply, subtract, divide, exponentiate]

In [2]:
add

StructuredTool(name='add', description="Add 'a' and 'b'. Both 'a' and 'b' are numbers.", args_schema=<class 'langchain_core.utils.pydantic.add'>, func=<function add at 0x107fb1080>)

In [3]:
print(f"{add.name=}\n{add.description}")

add.name='add'
Add 'a' and 'b'. Both 'a' and 'b' are numbers.


In [6]:
add.args_schema.model_json_schema()

{'description': "Add 'a' and 'b'. Both 'a' and 'b' are numbers.",
 'properties': {'a': {'title': 'A', 'type': 'number'},
  'b': {'title': 'B', 'type': 'number'}},
 'required': ['a', 'b'],
 'title': 'add',
 'type': 'object'}

In [9]:
exponentiate.args_schema.model_json_schema()

{'description': "Raise 'a' to the power of 'b'. Both 'a' and 'b' are numbers.",
 'properties': {'a': {'title': 'A', 'type': 'number'},
  'b': {'title': 'B', 'type': 'number'}},
 'required': ['a', 'b'],
 'title': 'exponentiate',
 'type': 'object'}

In [14]:
import json

llm_output_string = "{\"a\": 5, \"b\": 2}"
llm_output_dict = json.loads(llm_output_string)
llm_output_dict

{'a': 5, 'b': 2}

In [15]:
exponentiate.func(**llm_output_dict)

25

Create agent

In [27]:
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

SYSTEM_PROMPT = """You are a helpful math assistant.
You can use tools to do calculations instead of doing them in your head.
Always explain what you are doing in the final answer.
"""

In [17]:
from langchain_ollama.chat_models import ChatOllama

model_name = "llama3.2"
llm = ChatOllama(model=model_name, temperature=0.0)

In [29]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm, 
    tools=tools, 
    system_prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer)

# 6. Shared config: this `thread_id` is your "session id"
config = {"configurable": {"thread_id": "kate_math"}}

In [30]:
result1 = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "Hi, my name is Kate."}
        ]
    },
    config=config,
)

print("---- Response 1 ----")
print(result1["messages"][-1].content)  # last assistant message


---- Response 1 ----
Hello Kate! It's nice to meet you. I'm here to help with any math-related questions or problems you may have. What can I assist you with today?


In [ ]:
result2 = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "What is 10 multiplied by 7?"}
        ]
    },
    config=config,
)

print("---- Response 2 ----")
print(result2["messages"][-1].content)

---- Response 2 ----
The answer to 10 multiplied by 7 is 70, Kate! I used a multiplication tool to calculate this for me. Let me know if you have any other questions or need help with anything else!


In [34]:
print("---- Full History ----")
for message in result2["messages"]:
    print(message)

---- Full History ----
content='Hi, my name is Kate.' additional_kwargs={} response_metadata={} id='a447c8a7-43f1-439f-9aa0-893dcf9f9a0a'
content='' additional_kwargs={} response_metadata={'model': 'llama3.2', 'created_at': '2025-12-09T10:07:09.489305Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3528992750, 'load_duration': 2636790667, 'prompt_eval_count': 455, 'prompt_eval_duration': 537773583, 'eval_count': 22, 'eval_duration': 246805044, 'logprobs': None, 'model_name': 'llama3.2', 'model_provider': 'ollama'} id='lc_run--60591810-a9e2-4b3b-beba-dbecfc579e0f-0' tool_calls=[{'name': 'add', 'args': {'a': 0, 'b': 0}, 'id': '7e25fceb-bb51-477c-a1be-1092c8b1d7b0', 'type': 'tool_call'}] usage_metadata={'input_tokens': 455, 'output_tokens': 22, 'total_tokens': 477}
content='0.0' name='add' id='9439a046-b6a7-4dfa-a3bd-10413609edb4' tool_call_id='7e25fceb-bb51-477c-a1be-1092c8b1d7b0'
content="Hello Kate! It's nice to meet you. I'm here to help with any math-related questions or pr